In [1]:
import os
import glob
import pandas as pd

In [2]:
dpath = '/Users/jaehyukpark/GitRepo/artist_network/data/'

In [63]:
# Generate the id matching table
id_list = [i for i in range(1, 1001)]
id_df = pd.DataFrame(id_list, columns=['artist_id'])
for i in range(1, 4):
    p_df = pd.read_csv(os.path.join(dpath, 'id_p{}.csv'.format(i)))
    id_df = pd.merge(id_df, p_df, on='artist_id', how='outer')
id_df = id_df[['p1_id', 'p2_id', 'p3_id']].dropna(how='all')
#id_df['artist_id'] = id_df.index + 1 
print(id_df.tail())

     p1_id  p2_id  p3_id
992    NaN  566.0  668.0
993    NaN  567.0  669.0
994    NaN    NaN  670.0
996    NaN  568.0  671.0
998    NaN    NaN  672.0


In [35]:
# Convert ids in edgelist files to unique artist ids
d_range = ['all']
for r in d_range:
    for p in range(1, 4):
        for f in glob.glob(os.path.join(dpath, '{}/p{}/*.dat'.format(r,p))):
            outf = f.replace('/{}/'.format(r), '/{}_unique_id/'.format(r))
            df = pd.read_csv(f, sep='\t', header=None)
            # Conversion for edgelist files
            if ('pro_' in f) or ('market_' in f):
                df.columns = ['n1', 'n2']
                for col in df.columns:
                    df[col] = pd.merge(df, id_df[['artist_id', 'p{}_id'.format(p)]], left_on=col, 
                                              right_on='p{}_id'.format(p), how='left')['artist_id']
            # Conversion for attribute files
            else:
                y_cols = ['y{}'.format(i) for i in range(1,6)]
                df.columns = y_cols
                df['p{}_id'.format(p)] = [i+1 for i in range(len(df))]
                df = pd.merge(id_df, df, on='p{}_id'.format(p), how='left')[y_cols]
                df = df.fillna(0)
            df.to_csv(outf, sep='\t', index=False, header=False)

In [50]:
# Test
df = pd.DataFrame()
for i in range(11, 16):
    fname = os.path.join(dpath, 'all/p3/market_all_20{}.dat'.format(i))
    ydf = pd.read_csv(fname, sep='\t', header=None)
    #ydf.columns= [str(i) for i in range(len(df.columns))]
    df = pd.concat([df, ydf], ignore_index=True)

In [61]:
id_df.tail()

,p1_id,p2_id,p3_id
992,NaN,566.0,668.0
993,NaN,567.0,669.0
994,NaN,NaN,670.0
996,NaN,568.0,671.0
998,NaN,NaN,672.0
